<a href="https://colab.research.google.com/github/ndow33/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module4-Hyperparameter-Tuning/LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# clean the data

In [40]:
# Load the data
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Lambda/NLP/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [41]:
# use .describe() to explore the data
df.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2,NaN,2,3,3,3,3,3,3,3,3,3,2,4,NaN,6531,2
top,8313-KTIHG,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,20.2,No
freq,1,3555,NaN,3641,4933,NaN,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,NaN,11,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN,NaN


In [42]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [43]:
# change totalcharges to a float
# check for nan's
df['TotalCharges'].isna().value_counts()

False    7043
Name: TotalCharges, dtype: int64

In [44]:
# replace the ' ' with 0.0 and turn the rest of the strings into floats
counter = 0

for x in df['TotalCharges']:
  if x == ' ':
    y = 0
    df['TotalCharges'][counter] = float(y)
    counter = counter + 1 
  else:
    y = float(x)
    df['TotalCharges'][counter] = y
    counter = counter + 1

type(df['TotalCharges'][488])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


float

In [45]:
type(df['TotalCharges'][480])

float

In [46]:
# check for nan's throughout the dataframe
features = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 
            'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 
            'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
            'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 
            'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
            
categoricals = ['gender', 'Partner', 'Dependents', 
            'PhoneService', 'MultipleLines', 'InternetService', 
            'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
            'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 
            'PaymentMethod', 'Churn']

for feature in features:
  print(f'{feature}: ', df[feature].isna().value_counts())

customerID:  False    7043
Name: customerID, dtype: int64
gender:  False    7043
Name: gender, dtype: int64
SeniorCitizen:  False    7043
Name: SeniorCitizen, dtype: int64
Partner:  False    7043
Name: Partner, dtype: int64
Dependents:  False    7043
Name: Dependents, dtype: int64
tenure:  False    7043
Name: tenure, dtype: int64
PhoneService:  False    7043
Name: PhoneService, dtype: int64
MultipleLines:  False    7043
Name: MultipleLines, dtype: int64
InternetService:  False    7043
Name: InternetService, dtype: int64
OnlineSecurity:  False    7043
Name: OnlineSecurity, dtype: int64
OnlineBackup:  False    7043
Name: OnlineBackup, dtype: int64
DeviceProtection:  False    7043
Name: DeviceProtection, dtype: int64
TechSupport:  False    7043
Name: TechSupport, dtype: int64
StreamingTV:  False    7043
Name: StreamingTV, dtype: int64
StreamingMovies:  False    7043
Name: StreamingMovies, dtype: int64
Contract:  False    7043
Name: Contract, dtype: int64
PaperlessBilling:  False    7043
N

# one hot encode the categorical variables

In [47]:
!pip install category_encoders==2.*
import category_encoders as ce

In [48]:
df['gender'][1] == 'Male'

True

In [49]:
# gender
counter = 0
for x in df['gender']:
  if x == 'Male':
    df['gender'][counter] = 0
    print(counter)
  if x == 'Female':
    df['gender'][counter] = 1
    print('female')
  counter = counter + 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Streaming output truncated to the last 5000 lines.
2044
female
female
2047
2048
female
female
female
female
2053
female
2055
2056
female
2058
2059
2060
2061
female
2063
female
female
2066
2067
2068
female
female
female
2072
2073
female
2075
female
female
2078
2079
2080
female
2082
female
2084
female
2086
female
female
female
female
female
2092
2093
female
2095
female
2097
2098
female
2100
female
2102
2103
female
female
2106
2107
female
2109
female
2111
female
2113
2114
female
2116
female
female
2119
female
2121
female
2123
2124
female
2126
2127
2128
2129
2130
2131
2132
female
female
female
2136
2137
female
female
female
2141
female
2143
2144
2145
female
2147
2148
female
2150
2151
female
female
female
female
female
2157
female
female
female
female
2162
2163
2164
female
female
2167
2168
female
female
female
female
2173
female
female
2176
2177
2178
female
2180
female
2182
2183
female
2185
2186
2187
female
2189
2190
female
2192
2193
2194
female
female
2197
female
2199
female
female
2202
22

In [50]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,0,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,0,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,0,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,1,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [51]:
df1 = df.copy()
df1.drop(columns='customerID')

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,0,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,0,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,0,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,1,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,1,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,1,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,0,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [52]:
enc = ce.OneHotEncoder(use_cat_names=True)
enc

OneHotEncoder(cols=None, drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=True,
              verbose=0)

In [53]:
gender = enc.fit_transform(df1['gender'])
gender

,gender_1.0,gender_0.0
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
...,...,...
7038,0,1
7039,1,0
7040,1,0
7041,0,1


In [56]:
gender_enc = gender['gender_0.0']

In [54]:
enc = ce.OneHotEncoder(use_cat_names=True)
partner = enc.fit_transform(df1['Partner'])
partner = partner['Partner_Yes']

In [59]:
enc = ce.OneHotEncoder(use_cat_names=True)
dependents = enc.fit_transform(df1['Dependents'])
dependents = dependents['Dependents_Yes']
dependents

0       0
1       0
2       0
3       0
4       0
       ..
7038    1
7039    1
7040    1
7041    0
7042    0
Name: Dependents_Yes, Length: 7043, dtype: int64

In [61]:
enc = ce.OneHotEncoder(use_cat_names=True)
phone = enc.fit_transform(df1['PhoneService'])
phone

,PhoneService_No,PhoneService_Yes
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
7038,0,1
7039,0,1
7040,1,0
7041,0,1


In [62]:
phone = phone['PhoneService_Yes']

In [64]:
enc = ce.OneHotEncoder(use_cat_names=True)
lines = enc.fit_transform(df1['MultipleLines'])
lines

,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0
...,...,...,...
7038,0,0,1
7039,0,0,1
7040,1,0,0
7041,0,0,1


In [66]:
lines = lines['MultipleLines_Yes']

In [68]:
enc = ce.OneHotEncoder(use_cat_names=True)
internet = enc.fit_transform(df1['InternetService'])
internet

,InternetService_DSL,InternetService_Fiber optic,InternetService_No
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,1,0
...,...,...,...
7038,1,0,0
7039,0,1,0
7040,1,0,0
7041,0,1,0


In [69]:
dsl = internet['InternetService_DSL']
optic = internet['InternetService_Fiber optic']
none = internet['InternetService_No']

In [71]:
enc = ce.OneHotEncoder(use_cat_names=True)
security = enc.fit_transform(df1['OnlineSecurity'])
security

,OnlineSecurity_No,OnlineSecurity_Yes,OnlineSecurity_No internet service
0,1,0,0
1,0,1,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
7038,0,1,0
7039,1,0,0
7040,0,1,0
7041,1,0,0


In [72]:
security = security['OnlineSecurity_Yes']

In [74]:
enc = ce.OneHotEncoder(use_cat_names=True)
backup = enc.fit_transform(df1['OnlineBackup'])
backup

,OnlineBackup_Yes,OnlineBackup_No,OnlineBackup_No internet service
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,0,1,0
...,...,...,...
7038,0,1,0
7039,1,0,0
7040,0,1,0
7041,0,1,0


In [75]:
backup = backup['OnlineBackup_Yes']

In [77]:
enc = ce.OneHotEncoder(use_cat_names=True)
protection = enc.fit_transform(df1['DeviceProtection'])
protection

,DeviceProtection_No,DeviceProtection_Yes,DeviceProtection_No internet service
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
7038,0,1,0
7039,0,1,0
7040,1,0,0
7041,1,0,0


In [78]:
protection = protection['DeviceProtection_Yes']

In [80]:
enc = ce.OneHotEncoder(use_cat_names=True)
support = enc.fit_transform(df1['TechSupport'])
support

,TechSupport_No,TechSupport_Yes,TechSupport_No internet service
0,1,0,0
1,1,0,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
7038,0,1,0
7039,1,0,0
7040,1,0,0
7041,1,0,0


In [81]:
support = support['TechSupport_Yes']

In [83]:
enc = ce.OneHotEncoder(use_cat_names=True)
streaming = enc.fit_transform(df1['StreamingTV'])
streaming

,StreamingTV_No,StreamingTV_Yes,StreamingTV_No internet service
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
7038,0,1,0
7039,0,1,0
7040,1,0,0
7041,1,0,0


In [84]:
streaming = streaming['StreamingTV_Yes']

In [86]:
enc = ce.OneHotEncoder(use_cat_names=True)
movies = enc.fit_transform(df1['StreamingMovies'])
movies

,StreamingMovies_No,StreamingMovies_Yes,StreamingMovies_No internet service
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
7038,0,1,0
7039,0,1,0
7040,1,0,0
7041,1,0,0


In [87]:
movies = movies['StreamingMovies_Yes']

In [89]:
enc = ce.OneHotEncoder(use_cat_names=True)
contract = enc.fit_transform(df1['Contract'])
contract

,Contract_Month-to-month,Contract_One year,Contract_Two year
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
7038,0,1,0
7039,0,1,0
7040,1,0,0
7041,1,0,0


In [90]:
contract = contract['Contract_Month-to-month']

In [92]:
enc = ce.OneHotEncoder(use_cat_names=True)
paperless = enc.fit_transform(df1['PaperlessBilling'])
paperless

,PaperlessBilling_Yes,PaperlessBilling_No
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0
...,...,...
7038,1,0
7039,1,0
7040,1,0
7041,1,0


In [93]:
paperless = paperless['PaperlessBilling_Yes']

In [95]:
enc = ce.OneHotEncoder(use_cat_names=True)
churn = enc.fit_transform(df1['Churn'])
churn

,Churn_No,Churn_Yes
0,1,0
1,1,0
2,0,1
3,1,0
4,0,1
...,...,...
7038,1,0
7039,1,0
7040,1,0
7041,0,1


In [96]:
churn = churn['Churn_Yes']

In [97]:
df1['gender'] = gend_enc
df1['Partner'] = partner
df1['Dependents'] = dependents
df1['PhoneService'] = phone
df1['MultipleLines'] = lines
df1['InternetService'] = none
df1['OnlineSecurity'] = security
df1['OnlineBackup'] = backup
df1['DeviceProtection'] = protection
df1['TechSupport'] = support
df1['StreamingTV'] = streaming
df1['StreamingMovies'] = movies
df1['Contract'] = contract # month to month or not
df1['PaperlessBilling'] = paperless
df1['Churn'] = churn



df1.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,0,1,0,1,0,0,0,0,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0,0,1,1,0,0,0,0,1,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,0,0,1,0,1,1,0,0,0,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,0,0,0,0,0,0,0,1,1,Electronic check,70.70,151.65,1


In [98]:
df1.describe()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.421837,0.216669,0.286668,0.344881,0.343888,0.290217,0.384353,0.387903,0.550192,0.592219,64.761692,0.265370
std,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.493888,0.412004,0.452237,0.475363,0.475038,0.453895,0.486477,0.487307,0.497510,0.491457,30.090047,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.500000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,70.350000,0.000000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,89.850000,1.000000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,118.750000,1.000000


# Create and fit a baseline Keras MLP model to the data.

In [99]:
df1.shape

(7043, 21)

In [120]:
y = df1['Churn']
X = df1.drop(columns=['Churn', 'customerID', 'PaymentMethod'])

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges
248,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,48.55,48.55
3594,0,0,0,0,9,0,0,0,0,1,0,0,0,1,1,0,40.75,359.4
3846,0,0,1,1,8,1,0,0,0,0,0,0,0,0,1,1,45.45,411.75
4881,1,0,1,0,28,1,0,0,0,0,0,1,0,0,1,0,76.55,2065.4
148,1,0,0,1,30,1,0,0,0,1,1,0,1,0,0,0,64.50,1929.95


In [122]:
X_train = np.asarray(X_train.values)
y_train = np.asarray(y_train.values)
X_test = np.asarray(X_test.values)
y_test = np.asarray(y_test.values)

In [123]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train[:10])

[[ 0.98940648 -0.43803705 -0.97199303 -0.65570515 -1.27333258  0.32891113
  -0.85366923 -0.52903004  1.58576796 -0.72155927 -0.73011042 -0.63335929
  -0.79088868 -0.79355955  0.9047611   0.83118972 -0.53388826 -0.98130154]
 [-1.01070694 -0.43803705 -0.97199303 -0.65570515 -0.94736037 -3.04033491
  -0.85366923 -0.52903004 -0.63060929  1.38588755 -0.73011042 -0.63335929
  -0.79088868  1.26014487  0.9047611  -1.20309476 -0.79265959 -0.84363692]
 [-1.01070694 -0.43803705  1.02881396  1.52507572 -0.9881069   0.32891113
  -0.85366923 -0.52903004 -0.63060929 -0.72155927 -0.73011042 -0.63335929
  -0.79088868 -0.79355955  0.9047611   0.83118972 -0.63673328 -0.82045293]
 [ 0.98940648 -0.43803705  1.02881396 -0.65570515 -0.17317636  0.32891113
  -0.85366923 -0.52903004 -0.63060929 -0.72155927 -0.73011042  1.57888265
  -0.79088868 -0.79355955  0.9047611  -1.20309476  0.39503447 -0.08810906]
 [ 0.98940648 -0.43803705 -0.97199303  1.52507572 -0.0916833   0.32891113
  -0.85366923 -0.52903004 -0.63060

In [136]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(inputs,)))
model.add(Dense(8, activation='relu')),
model.add(Dense(8, activation='relu')),
model.add(Dense(8, activation='relu')),
model.add(Dense(1))

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Epoch 1/75
564/564 [==============================] - 1s 2ms/step - loss: 0.6425 - acc: 0.7270 - val_loss: 0.5062 - val_acc: 0.7559
Epoch 2/75
564/564 [==============================] - 1s 1ms/step - loss: 0.4699 - acc: 0.7733 - val_loss: 0.4505 - val_acc: 0.7857
Epoch 3/75
564/564 [==============================] - 1s 1ms/step - loss: 0.4458 - acc: 0.7925 - val_loss: 0.4383 - val_acc: 0.7963
Epoch 4/75
564/564 [==============================] - 1s 1ms/step - loss: 0.4417 - acc: 0.7957 - val_loss: 0.4347 - val_acc: 0.8034
Epoch 5/75
564/564 [==============================] - 1s 1ms/step - loss: 0.4448 - acc: 0.7952 - val_loss: 0.4389 - val_acc: 0.7999
Epoch 6/75
564/564 [==============================] - 1s 1ms/step - loss: 0.4359 - acc: 0.7952 - val_loss: 0.4323 - val_acc: 0.8091
Epoch 7/75
564/564 [==============================] - 1s 1ms/step - loss: 0.4390 - acc: 0.8021 - val_loss: 0.4320 - val_acc: 0.8091
Epoch 8/75
564/564 [==============================] - 1s 1ms/step - loss: 0.

# tune these hyperparameters
- batch_size
- training epochs
- optimizer
- learning rate (if applicable to optimizer)
- momentum (if applicable to optimizer)
- activation functions
- network weight initialization
- dropout regularization
- number of neurons in the hidden layer




In [127]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [147]:
from tensorflow.keras.optimizers import Adam
# Function to create model, required for KerasClassifier
def create_model(lr):
  # Create optimizer
  adam = Adam(lr=lr)
    # create model
  model = Sequential()
  model.add(Dense(8, activation='relu', input_shape=(inputs,)))
  model.add(Dense(1))
    # Compile model
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [148]:
# define the grid search parameters
param_grid = {'batch_size': [2, 4, 8, 16, 32],
              'epochs': [10, 20, 30, 40, 50, 100, 200],
              'lr': [.001, .01, .1]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [143]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7937557220458984 using {'batch_size': 2, 'epochs': 20}
Means: 0.7937557220458984, Stdev: 0.013819692511555837 with: {'batch_size': 2, 'epochs': 20}
Means: 0.7923357248306274, Stdev: 0.01326668134737322 with: {'batch_size': 4, 'epochs': 20}
Means: 0.7791975140571594, Stdev: 0.012780789643641896 with: {'batch_size': 8, 'epochs': 20}
Means: 0.7673045516014099, Stdev: 0.00897979144058996 with: {'batch_size': 16, 'epochs': 20}
Means: 0.7532806158065796, Stdev: 0.021102637293348752 with: {'batch_size': 32, 'epochs': 20}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?